In [ ]:
from IPython.display import display
import _functions

# #Initialize client
client = _functions.AsyncRequestClient('', '')

# Initialize the set_widgets list
token_input, url_input, query, field_id, bool_toggle, dropdown = _functions.create_input_widgets()
display(token_input, dropdown, url_input, query, field_id, bool_toggle)


In [ ]:
# Get Data
await _functions.process_annotations(client, token_input, url_input, query, field_id, bool_toggle, dropdown)